In [ ]:
# ============================================================
# 🔎 Substructure Matching Pipeline (with start_id option)
#
# This block compares two datasets:
# - df1: Compounds with "ID" and "Smiles" (can be a previous 'results.xlsx')
# - df2: Bioactive ChEMBL substructures ("SMILES", "preferred target")
#
# Note:
# The "magic rings" or Bioactive ChEMBL rings were extracted from this article:
# https://doi.org/10.1021/acs.jcim.1c00761
# Only those substructures with a defined specific target were considered.
# The targets used in this dataset are:
#   protease
#   kinase
#   GPCR
#   ion channel
#   nuclear receptor
#   transporter
#   epigenetic
# The substructures and their targets are provided in the file 'magic rings.xlsx'.
#
# This script can be used to update an existing compound database.
# An example of the expected structure of df1 (input file) of a database to update is as follows:
#
# ID        Smiles        Bioactive ChEMBL ring 1  Target of bioactive ChEMBL ring 1  Bioactive ChEMBL ring 2  Target of bioactive ChEMBL ring 2
# CMPD001   CCO           C1CCO                     Protease                              CCN(CC)C                  Kinase
# CMPD002   C1CCO1        COC                       GPCR                                  C1=CC=CC=C1                Ion channel
# CMPD003   CCN(CC)C      CC=O                      Nuclear receptor                      <empty>                    <empty>
# CMPD004   COC           <empty>                   <empty>                               <empty>                    <empty>
# CMPD005   CC(=O)O       <empty>                   <empty>                               <empty>                    <empty>
#
# For example, if you want to resume processing from compound CMPD004, 
# you should set the variable start_id = "CMPD004" in the section below.
#
# Workflow:
# 1. Convert SMILES to RDKit Mol objects if not already present.
# 2. For each compound in df1 (starting from start_id if given), check substructure matches with df2.
# 3. If match found, create new columns:
#    - "Bioactive ChEMBL ring N" → substructure SMILES
#    - "Target of bioactive ChEMBL ring N" → corresponding target
# 4. Ensures empty cells are blank, never NaN.
# 5. Real-time progress with last processed ID.
# ============================================================


import pandas as pd
from rdkit import Chem

# ---------------- Optional: start calculation from a specific ID ----------------
# ID must be provided as a string, e.g., 'LANaPDB14'.
# You can use any ID present in your input table (see the example table above with CMPD001, CMPD002, etc.).
# If start_id is None or not found in the DataFrame, processing starts from the first row.
start_id = None  # <-- change as needed, or set to None to start from first row

# ---------------- Load data ----------------
# df1: your compound dataset (with at least 'ID' and 'Smiles' columns)
# Replace 'input_file.xlsx' with the actual filename you want to process
df1 = pd.read_excel('input_file.xlsx')  

# df2: contains the magic rings / Bioactive ChEMBL rings and their targets
# Must be in 'magic rings.xlsx' with columns 'SMILES' and 'preferred target'
df2 = pd.read_excel('magic rings.xlsx')

# ---------------- Convert SMILES to Mol ----------------
def convert_to_mol(smiles):
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

# Only create 'Mol1' if it doesn't exist yet
if 'Mol1' not in df1.columns:
    df1['Mol1'] = df1['Smiles'].apply(convert_to_mol)

# Convert SMILES in df2 to Mol objects
if 'Mol2' not in df2.columns:
    df2['Mol2'] = df2['SMILES'].apply(convert_to_mol)

# ---------------- Determine starting index ----------------
if start_id is not None and start_id in df1['ID'].values:
    start_idx = df1.index[df1['ID'] == start_id][0]
elif start_id is not None:
    print(f"⚠️ start_id '{start_id}' not found. Starting from the first row.")
    start_idx = 0
else:
    start_idx = 0

# ---------------- Substructure search ----------------
for idx1 in range(start_idx, len(df1)):
    row1 = df1.loc[idx1]
    mol1 = row1['Mol1']
    if mol1 is None:
        continue

    match_count = 1
    for idx2, row2 in df2.iterrows():
        mol2 = row2['Mol2']
        if mol2 is None:
            continue

        try:
            if mol1.HasSubstructMatch(mol2):
                bio_col = f'Bioactive ChEMBL ring {match_count}'
                target_col = f'Target of bioactive ChEMBL ring {match_count}'

                # Create columns if they don't exist
                if bio_col not in df1.columns:
                    df1[bio_col] = [""] * len(df1)
                if target_col not in df1.columns:
                    df1[target_col] = [""] * len(df1)

                # Assign values
                df1.at[idx1, bio_col] = str(row2['SMILES']) if row2['SMILES'] else ""
                df1.at[idx1, target_col] = str(row2['preferred target']) if row2['preferred target'] else ""

                match_count += 1
        except Exception:
            continue

    # Print last successfully processed ID
    print(f"\r✅ Last processed ID: {row1['ID']}", end='', flush=True)

# ---------------- Cleanup ----------------
df1.drop(columns=['Mol1'], inplace=True)

# ---------------- Save results ----------------
df1.to_excel('results.xlsx', index=False)

# ---------------- Optional: display DataFrame in Jupyter ----------------
# Uncomment the following line to view the DataFrame in the notebook
# display(df1)


In [ ]:
# ---------------- Optional: display the processed DataFrame in Jupyter ----------------
# Note: df1 is the output saved in 'results.xlsx'.
# Uncomment the following line to view it in the notebook
#display(df1)


In [ ]:
# ============================================================
# 🖼 Generate RDKit Molecule Images for Visualization (Jupyter only)
#
# For all columns in df1 starting with "Bioactive ChEMBL ring":
# 1. Convert the SMILES in each column to RDKit Mol objects (safe parsing).
# 2. Add a new column "<column> structures" containing Mol objects.
# 3. In Jupyter, these Mol objects render automatically as molecule drawings
#    (if PandasTools is configured).
# 4. Original SMILES data remains unchanged.
# ============================================================

import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

# ---------------- Identify bioactive columns ----------------
bioactive_cols = [
    col for col in df1.columns
    if col.startswith("Bioactive ChEMBL ring") and "structures" not in col
]

# ---------------- Safe converter for SMILES ----------------
def safe_mol(smiles):
    if pd.isna(smiles):   # handle NaN
        return None
    if not isinstance(smiles, str):  # handle floats or non-string values
        return None
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

# ---------------- Convert SMILES to Mol objects ----------------
for col in bioactive_cols:
    new_col = f"{col} structures"
    df1[new_col] = df1[col].apply(safe_mol)

# ---------------- Enable visualization in Jupyter ----------------
PandasTools.RenderImagesInAllDataFrames(True)

In [ ]:
# ---------------- Optional: display entire DataFrame with molecule images ----------------
# Uncomment the following line to show the DataFrame in Jupyter
#display(df1)

In [ ]:
# ============================================================
# 🔄 Stack Bioactive ChEMBL Ring Columns
#
# This block takes all numbered "Bioactive ChEMBL ring N" columns
# in df1 and stacks them into a single column in a new long-format
# DataFrame df2, keeping the corresponding target and structure
# for each ring. Empty entries are removed.
# ============================================================

import pandas as pd

# Assuming your original df1 is already defined

# Create a new list to store rows for the new dataframe
new_data = []

# Iterate over numbered columns
for i in range(1, 3):  # Adjust 3 to the max number of ring columns you have
    bioactive_col = f'Bioactive ChEMBL ring {i}'
    target_col = f'Target of bioactive ChEMBL ring {i}'
    structure_col = f'Bioactive ChEMBL ring {i} structures'

    # Only proceed if all these columns exist
    if bioactive_col in df1.columns and target_col in df1.columns and structure_col in df1.columns:
        for index, row in df1.iterrows():
            new_data.append({
                'ID': row['ID'],
                'Bioactive ChEMBL ring': row[bioactive_col],
                'Target of bioactive ChEMBL ring': row[target_col],
                'Bioactive ChEMBL ring structures': row[structure_col]
            })

# Create the new long-form dataframe
df2 = pd.DataFrame(new_data)

# Remove rows where 'Bioactive ChEMBL ring' is empty
df2 = df2.dropna(subset=['Bioactive ChEMBL ring'])

In [ ]:
# ---------------- Optional: display DataFrame in Jupyter ----------------
# Uncomment the following line to view the DataFrame in the notebook
#display(df2)

In [ ]:
# ============================================================
# 📊 Calculate Counts and Percentages for Bioactive Rings & Targets
# (Three summary sections: Molecule Summary, Target Summary, Bioactive Ring Summary)
# ============================================================

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display, HTML
from io import BytesIO
import base64
import os

# ---------------- Identify all Bioactive ChEMBL ring columns ----------------
bioactive_cols = [col for col in df1.columns if col.startswith("Bioactive ChEMBL ring") and "structures" not in col]
target_cols = [col for col in df1.columns if col.startswith("Target of bioactive ChEMBL ring")]

# ---------------- Stack all rings into long-form df2 ----------------
new_data = []
for col in bioactive_cols:
    target_col = f"Target of {col}" if f"Target of {col}" in df1.columns else None
    for idx, row in df1.iterrows():
        ring = row[col]
        target = row[target_col] if target_col else ""
        if pd.notna(ring) and ring != "":
            new_data.append({
                'ID': row['ID'],
                'Bioactive ChEMBL ring': ring,
                'Target of Bioactive ChEMBL ring': target
            })

df2 = pd.DataFrame(new_data)

# ---------------- Count molecules with at least N rings ----------------
rings_per_molecule = df2.groupby('ID')['Bioactive ChEMBL ring'].count()

# Include molecules with 0 rings
all_ids = set(df1['ID'])
ids_with_rings = set(rings_per_molecule.index)
ids_with_0 = all_ids - ids_with_rings

zero_series = pd.Series(0, index=list(ids_with_0))
rings_per_molecule = pd.concat([rings_per_molecule, zero_series])

max_rings = rings_per_molecule.max()

# Create Molecule Summary table
summary_data = []
total_molecules = len(df1)

for n in range(0, max_rings + 1):
    if n == 0:
        count = (rings_per_molecule == 0).sum()
        summary_data.append({
            "N rings (≥N)": "0 rings",
            "Molecule count": count,
            "Percentage of input molecules (%)": round(count / total_molecules * 100, 2)
        })
    else:
        count = (rings_per_molecule >= n).sum()
        summary_data.append({
            "N rings (≥N)": f"≥{n} rings",
            "Molecule count": count,
            "Percentage of input molecules (%)": round(count / total_molecules * 100, 2)
        })

df_summary = pd.DataFrame(summary_data)

# ➕ Add total row for display only
df_summary_display = df_summary.copy()
df_summary_display.loc[len(df_summary_display)] = {
    "N rings (≥N)": "TOTAL",
    "Molecule count": df_summary_display["Molecule count"].sum(),
    "Percentage of input molecules (%)": round(df_summary_display["Percentage of input molecules (%)"].sum(), 2)
}

# ---------------- Save Molecule Summary as separate Excel ----------------
molecule_summary_file = 'molecule_summary.xlsx'
df_summary.to_excel(molecule_summary_file, index=False)

# ---------------- Calculate unique ring counts ----------------
ring_counts = df2['Bioactive ChEMBL ring'].value_counts(dropna=True).reset_index()
ring_counts.columns = ['Bioactive ChEMBL ring', 'Bioactive ChEMBL ring count']
df_counts = ring_counts.copy()

# Convert SMILES to Mol objects for display only
def safe_mol(smiles):
    if not smiles:
        return None
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

df_counts['ROMol'] = df_counts['Bioactive ChEMBL ring'].apply(safe_mol)

# Convert Mol to HTML images
def mol_to_img_html(mol, size=(150,150)):
    if mol is None:
        return ""
    img = Draw.MolToImage(mol, size=size)
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    b64 = base64.b64encode(buffer.getvalue()).decode()
    return f'<img src="data:image/png;base64,{b64}"/>'

df_counts['Bioactive ChEMBL ring structures'] = df_counts['ROMol'].apply(mol_to_img_html)
df_counts.drop(columns=['ROMol'], inplace=True)

# Calculate percentages
total_valid_count = df_counts['Bioactive ChEMBL ring count'].sum()
df_counts['Bioactive ChEMBL ring percentage'] = (
    df_counts['Bioactive ChEMBL ring count'] / total_valid_count * 100
).round(2)

# ➕ Add total row for display only
df_counts_display = df_counts.copy()
df_counts_display.loc[len(df_counts_display)] = {
    "Bioactive ChEMBL ring": "TOTAL",
    "Bioactive ChEMBL ring count": df_counts_display["Bioactive ChEMBL ring count"].sum(),
    "Bioactive ChEMBL ring structures": "",
    "Bioactive ChEMBL ring percentage": round(df_counts_display["Bioactive ChEMBL ring percentage"].sum(), 2)
}

# ---------------- Save Bioactive Rings to Excel (without structures, without TOTAL row) ----------------
counts_file = 'counts_rings.xlsx'
with pd.ExcelWriter(counts_file, engine='openpyxl') as writer:
    df_counts.drop(columns=['Bioactive ChEMBL ring structures']).to_excel(writer, sheet_name="Bioactive Ring Counts", index=False)
    # Optional: could also include df_summary if desired
    df_summary.to_excel(writer, sheet_name="Molecule Summary", index=False)

# ---------------- Targets: counts and percentages (Excel + display) ----------------
df_targets_stacked = df1[target_cols].stack()
df_targets_stacked = df_targets_stacked.astype(str).str.strip()
df_targets_stacked = df_targets_stacked[~df_targets_stacked.isin(["", "nan", "None"])]

if not df_targets_stacked.empty:
    target_counts = df_targets_stacked.value_counts().reset_index()
    target_counts.columns = ['Target', 'Count']
    total_targets = target_counts['Count'].sum()
    target_counts['Percentage'] = (target_counts['Count'] / total_targets * 100).round(2)
else:
    target_counts = pd.DataFrame(columns=['Target', 'Count', 'Percentage'])

# ➕ Add TOTAL row for display only
target_counts_display = target_counts.copy()
if not target_counts_display.empty:
    target_counts_display.loc[len(target_counts_display)] = {
        "Target": "TOTAL",
        "Count": target_counts_display["Count"].sum(),
        "Percentage": round(target_counts_display["Percentage"].sum(), 2)
    }

# Save targets to Excel WITHOUT TOTAL row
targets_file = 'counts_targets_of_the_bioactive_rings.xlsx'
target_counts.to_excel(targets_file, index=False)

# ---------------- Print outputs ----------------
print(f"Total molecules (from original df1 input): {total_molecules}")
print(f"Number of unique Bioactive ChEMBL rings: {df_counts['Bioactive ChEMBL ring'].nunique()}")

# 📊 Molecule summary
print("\n📊 Molecule counts by number of Bioactive ChEMBL rings per compound")
print(f"ℹ️ This result will be saved in '{molecule_summary_file}'")
display(df_summary_display)

print(
    "\n⚠️ Note: The TOTAL row can exceed the number of input molecules and 100%. "
    "This happens because the same molecule can contribute to multiple rows. "
    "For example, a molecule with 2 bioactive rings is counted once in '≥1 rings', "
    "again in '≥2 rings', and so on. "
    "However, the sum of the first two rows ('0 rings' and '≥1 rings') must equal the total number "
    "of input compounds and 100% of the dataset."
)

# 📊 Target summary
print("\n📊 Target counts and percentages")
print(f"ℹ️ These results will be saved in '{targets_file}'")
print("   Molecules without any Bioactive ChEMBL ring are not included in this calculation.")
display(target_counts_display)

# ---------------- Explanation about Bioactive Ring percentages ----------------
print(
    f"\n📊 Bioactive ChEMBL ring percentage\n"
    f" - Number of unique Bioactive ChEMBL rings: {df_counts['Bioactive ChEMBL ring'].nunique()}\n"
    " - Percentage of each unique bioactive ring based on 'Bioactive ChEMBL ring count'.\n"
    " - Calculation also includes all bioactive rings from molecules that have 2 or more rings.\n"
    " - Rows with 'None' Bioactive ChEMBL ring are excluded from the calculation."
)

# 📊 Bioactive ring structures
print(f"\nℹ️ This result will be saved in '{counts_file}'")
display(HTML(df_counts_display.to_html(escape=False)))

# ---------------- Confirm file creation ----------------
print("\n📁 Saved files:")
for file in [counts_file, molecule_summary_file, targets_file]:
    if os.path.exists(file):
        print(f" - {file}")
    else:
        print(f" - {file} was not created!")